# Parallel computing with Pytorch in deep learning

author: Jing Zhang    
e-mail: zhangjingnm@hotmail.com    
date: 2024-09    
reference: http://www.idris.fr/eng/jean-zay/gpu/jean-zay-gpu-torch-multi-eng.html